# 0. selenium 설치

In [12]:
!pip install selenium

In [13]:
!pip install chromedriver_autoinstaller

# 1. 데이터 불러오기

In [14]:
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함


from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [15]:
# 선수명 csv파일 불러오기
data = pd.read_csv('./data/player.csv')
data

,id,play_name
0,0,데이비드 시먼
1,1,앨런 시어러
2,2,로비 파울러
3,3,로이 킨
4,4,라이언 긱스
...,...,...
59046,59046,A. 크나우프
59047,59047,K. 야키치
59048,59048,M. 구스토
59049,59049,G. 마마르다슈빌리


# 2. 데이터 크롤링

In [63]:
## 크롬 윈도우 설정
# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=600,1400")

# 실행할 시간 랜덤값 지정
A = np.random.randint(1,3)

## 2-1. 선수 데이터 크롤링


- 선수명
- 급여
- 키
- 몸무게
- 체형
- 개인기
- 주발
- 오버롤 + 전체 스텟
- 시세(일별 금액)
- 클럽경력
- 팀컬러 (팀, 특성, 강화)

*시세의 경우 각 강화 등급별 크롤링! (나머지 값은 후처리로 기입)

In [10]:
# 선수 데이터 크롤링
'''
# 크롤링한 결과를 저장할 리스트
name_list = []
salary_list = []
height_list = []
weight_list = []
skill_list = []
prefer_foot = []
overall_list = []
stat_list = []
price_list = []
club_list = []
teamcolor_list = []





for player_name in tqdm(data['play_name'][:5]):
    try:   
        chrome_path = chromedriver_autoinstaller.install()
        driver = webdriver.Chrome(chrome_path, options=options)
        driver.get('https://fifaonline4.nexon.com/datacenter')
        time.sleep(A)
        
        # 선수명 입력
        driver.find_elements_by_css_selector('#searchName').click()
        driver.find_element_by_name('strPlayerName').send_keys(f'{player_name}')
        driver.find_elements_by_css_selector('#form1 > div.search_panel > div.search_input_submit > button.btn_search').click()
        time.sleep(A)
        
    except:
        print('error')
        break
        
'''


  0%|          | 0/5 [00:00<?, ?it/s]

error


## 2-2. 팀 컬러 크롤링



- 팀 컬러 명
- 단계
- 적용 효과

*특성 팀 컬러의 경우'적용 선수' 추가

In [76]:
## 팀 컬러 목록 크롤링
# 팀 컬러 타입 리스트
teamcolor_club = []
teamcolor_nation = []
teamcolor_reinforce = []
teamcolor_relation = []
teamcolor_special =[]


# 홈페이지 열기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)
time.sleep(A)
        
# 팀 컬러 정보 목록 저장
for x in tqdm(range(2,7)):
        driver.find_element_by_xpath(f'//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[{x}]/label').click()
        
        ## 클럽
        if x == 2:
                driver.find_element_by_class_name('btn_search').click()
                time.sleep(A)

                locates = 'div.name'
                elements = driver.find_elements_by_css_selector(locates)

                for element in elements:
                        teamcolor_club.append(element.text)
                
                
        
        elif x == 3:
                driver.find_element_by_class_name('btn_search').click()
                time.sleep(A)

                locates = 'div.name'
                elements = driver.find_elements_by_css_selector(locates)

                for element in elements:
                        teamcolor_nation.append(element.text)
        
        elif x == 4:
                driver.find_element_by_class_name('btn_search').click()
                time.sleep(A)

                locates = 'div.name'
                elements = driver.find_elements_by_css_selector(locates)

                for element in elements:
                        teamcolor_reinforce.append(element.text)
                        
        elif x == 5:
                driver.find_element_by_class_name('btn_search').click()
                time.sleep(A)

                locates = 'div.name'
                elements = driver.find_elements_by_css_selector(locates)

                for element in elements:
                        teamcolor_relation.append(element.text)
                        
        elif x == 6:
                driver.find_element_by_class_name('btn_search').click()
                time.sleep(A)

                locates = 'div.name'
                elements = driver.find_elements_by_css_selector(locates)

                for element in elements:
                        teamcolor_special.append(element.text)
        
        else:
                print('not in range')
        
        driver.find_element_by_class_name('btn_reset').click()
        time.sleep(A)
        

  0%|          | 0/5 [00:00<?, ?it/s]

In [85]:
# 팀 컬러 타입 모음
teamcolor_type = [teamcolor_club,teamcolor_nation,teamcolor_reinforce,teamcolor_relation,teamcolor_special]

for x in teamcolor_type:
    print(len(x))

192
57
3
206
8


In [131]:
## 팀 컬러 타입 별 적용 조건 & 효과 저장
# 홈페이지 열기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)
time.sleep(A)

for teamcolor in teamcolor_type:
   # 크롤링한 결과를 저장할 리스트
   step_1_require_player_nums = []
   step_2_require_player_nums = []
   step_3_require_player_nums = []
   step_4_require_player_nums = []
   step_1_effects = []
   step_2_effects = []
   step_3_effects = []
   step_4_effects = []
   
   for x in tqdm(teamcolor[:2]):
      # 팀 컬러 타입 선택
      if teamcolor == teamcolor_club:
         driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[2]/label').click()
      elif teamcolor == teamcolor_nation:
         driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[3]/label').click()
      elif teamcolor == teamcolor_reinforce:
         driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[4]/label').click() 
      elif teamcolor == teamcolor_relation:
         driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]/label').click() 
      elif teamcolor == teamcolor_special:
         driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[6]/label').click()  

      # 팀 컬러 입력 & 검색
      driver.find_element_by_class_name('ui-autocomplete-input').click()
      driver.find_element_by_class_name('ui-autocomplete-input').send_keys(x)
      driver.find_element_by_class_name('btn_search').click()
      time.sleep(A)

      # 세부 설명 들어가기
      driver.find_element_by_class_name('name').click()
      driver.find_element_by_class_name('btn_detail_link').click()
      time.sleep(A) 

      # # 단계 별 적용 조건 & 적용 효과 저장
      # for num in range(1,5):
      #     require_player_num = driver.find_element_by_css_selector(f'#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv{num} > div.lv_content > div.desc').text()
      #     application_effect = driver.find_element_by_css_selector(f'#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv{num} > div.lv_content > div.ap_list').text()
         
      #     # 1단계
      #     if num == 1:
      #         step_1_require_player_nums.append(require_player_num)
      #         step_1_effects.append(application_effect)

      #     # 2단계
      #     elif num == 2: 
      #         step_2_require_player_nums.append(require_player_num)
      #         step_2_effects.append(application_effect)
               
               
               

      #     # 3단계
      #     elif num == 3:
      #         step_3_require_player_nums.append(require_player_num)
      #         step_3_effects.append(application_effect)
               
               
               
         
      #     # 4단계
      #     elif num == 4:
      #         step_4_require_player_nums.append(require_player_num)
      #         step_4_effects.append(application_effect)
         
      #     else:
      #         print('error')
      
      # 세부 설명 나오기
      driver.find_element_by_class_name('btn_close').click()
      time.sleep(A)

      # 검색어 초기화 
      driver.find_element_by_class_name('btn_reset').click()
      time.sleep(A)



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

ElementClickInterceptedException: Message: element click intercepted: Element <label for="search_value3_3">...</label> is not clickable at point (537, 889). Other element would receive the click: <span class="ico ico_btn_top">...</span>
  (Session info: chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x0000000103277938 chromedriver + 4745528
1   chromedriver                        0x00000001031f9443 chromedriver + 4228163
2   chromedriver                        0x0000000102e5cb68 chromedriver + 441192
3   chromedriver                        0x0000000102ea1ceb chromedriver + 724203
4   chromedriver                        0x0000000102e9f4ed chromedriver + 713965
5   chromedriver                        0x0000000102e9ca34 chromedriver + 703028
6   chromedriver                        0x0000000102e9b78d chromedriver + 698253
7   chromedriver                        0x0000000102e8df61 chromedriver + 642913
8   chromedriver                        0x0000000102ebad52 ch

In [80]:
step_1_effects

[]

#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv1 > div.lv_content > div.ap_list

#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv2 > div.lv_content > div.ap_list

In [ ]:
'''
find_elements_by_tag_name:          Copy Element의 tag
find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
find_elements_by_id:                Copy Element의 id attribute
find_elements_by_class_name:        Copy Element의 class attribute
find_elements_by_link_text:         Copy Element의 text
find_elements_by_partial_link_text: Copy Element의 text중 일부
find_elements_by_css_selector:      Copy selector


'요소' 클릭:        	.click('')   
'요소' 더블 클릭:	     .double_click('')   
'텍스트' 입력:	         .send_keys('')  
'요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
'요소'로 마우스 이동:	  .move_to_element('')
스크롤 내리기:	         .execute_script("window.scrollto();")
실행:                  .perform()



'''

In [ ]:
#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv4 > div.lv_content > div.desc

#teamcolorPop > div > div.wrap > div.content_header > div.content > div.level.lv4 > div.lv_content > div.ap_list